# Wstęp do Uczenia Maszynowego - Projekt 1
## Autorzy: Katarzyna Solawa, Jan Smoleń


Tematem naszego projektu jest przewidywanie przynależności partyjnej członka Izby Reprezentantów amerykańskiego kongresu w 1986 roku na podstawie dokonanych przez niego wyborów podczas głosowań. Naszym zbiorem danych jest ramka zawierająca dane o przynależności partyjnej poszczególnych reprezentantów i ich głosach podczas 16 kluczowych w tym roku głosowań. 

# Importy

In [ ]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn.metrics
import random
from sklearn import manifold
random.seed(42)
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
import xgboost as xgb
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
import sklearn.metrics as metrics
import statistics

# EDA

In [ ]:
df=pd.read_csv("congressional_voting_dataset.csv")

In [ ]:
df.info()

In [ ]:
df.head()

## Objaśnienie zmiennych
Kolumny 0-15 zawierają wyniki głosowań na tematy skrótowo opisane w nazwach kolumn. Każdy rząd odpowiada jednemu reprezentantowi. Możliwe wartości: <br>
**y** - głos na tak <br>
**n** - głos na nie <br>
**?** - brak głosu - niewzięcie udziału w głosowaniu lub wstrzymanie się od głosu <br>
Ostatnia kolumna zawiera informacje o przynależności partyjnej reprezentanta - **republican** albo **democrat**. W naszej ramce danych nie występuje bezpośrednio problem braku danych, ale zapewne będzie trzeba jakoś rozwiązać kwestię wartości **?**.

In [ ]:
df.describe()

In [ ]:
labels=["y", "n", "?"]
fig, axs = plt.subplots(ncols=2, nrows=8, figsize=(16, 32))
for i in range(len(df.columns)-1):
    col=df.columns[i]
    tmp=df[[col, "political_party"]].groupby(["political_party", col]).size().tolist()
    r, c= i//2, i%2
    axs[r,c].bar(labels, list(reversed(tmp[0:3])), label='democrat', color="blue")
    axs[r,c].bar(labels, list(reversed(tmp[3:6])), bottom=list(reversed(tmp[0:3])),
       label='republican', color="red")
    axs[r,c].legend()
    axs[r,c].set_title(col)

Obie partie głosowały podobnie na `water_project_cost_sharing` oraz `imigration`(lecz u demokratów przeważa `no`, a u republikan `yes`)
Widoczna róznica głosów dla:
- `adoption_of_the_budget_resolution`(r-no, d-yes)
- `physician_fee_freeze`(r-yes, d-no)
- `el_salvador_aid`(r-yes, d-no)
- `education_spending`(r-yes, d-no)

In [ ]:
df=df.replace("n", 0)
df=df.replace("y", 1)
df=df.replace("?",  0.5)   #rozwiązanie tymczasowe
df=df.replace("republican", 0)
df=df.replace("democrat",  1)
plt.figure(figsize=(10,10))
sns.heatmap(df.corr(), annot=True, annot_kws={'size': 8}, fmt='.2f')
plt.show()

Jak widzimy, poziom korelacjii pomiędzy głosem a partią bardzo się różni w zależności od tematu głosowania - dla głosowania **water_project_cost_sharing** związek praktycznie nie istnieje, a dla **physician_fee_freeze** jest bardzo duży.

Spróbujemy teraz zobaczyć, na ile głosy poszczególnych reprezentantów przypominają głosy innych członków tej samej partii - w tym celu przekształcimy zapisy głosowań poszczególnych członków na wektory i policzymy odległości pomiędzy każdą parą. 

In [ ]:
adist=sklearn.metrics.pairwise_distances(df.drop(["political_party"], axis=1))
adist

Użyjemy teraz funkcji z pakietu manifold żeby przekształcić ramkę zawierającą wzajemne odległości na zbiór współrzędnych na dwuwymiarowej płaszczyźnie. Jest to rzut, który próbuje przekształcić wielowymiarowe zależności na płaszczyznę 2D. 

In [ ]:
df["political_party"]=df["political_party"].replace(0, "republican")
df["political_party"]=df["political_party"].replace(1, "democrat")
adist=np.array(adist)
mds = manifold.MDS(n_components=2, dissimilarity="precomputed", random_state=6)
results = mds.fit(adist)
coords = results.embedding_
fig, ax = plt.subplots(figsize=(10,10))
sns.scatterplot(
    coords[:, 0], coords[:, 1], marker = 'o', hue=df["political_party"], palette=["red", "blue"]
    )
ax.set_title("Voting pattern similarity")

Dodatkowo sprawdźmy czy któraś z parti ma skołonność do głosowania na tak lub nie.

In [ ]:
df=pd.read_csv("congressional_voting_dataset.csv")
democrat_df = df[df['political_party'] == 'democrat']
republican_df = df[df['political_party'] == 'republican']

In [ ]:
tak = 0
nie = 0
brak = 0
for i in range(0,15):
    tak += (democrat_df[democrat_df.columns[i]] == "y").sum()
    nie += (democrat_df[democrat_df.columns[i]] == "n").sum()
    brak += (democrat_df[democrat_df.columns[i]] == "?").sum()

labels = ['Yes', 'No', '?']
sizes = [tak, nie,brak]
plt.title("Democrat")
plt.bar(labels, sizes)
plt.show()

In [ ]:
tak = 0
nie = 0
brak = 0
for i in range(0,15):
    tak += (republican_df[republican_df.columns[i]] == "y").sum()
    nie += (republican_df[republican_df.columns[i]] == "n").sum()
    brak += (republican_df[republican_df.columns[i]] == "?").sum()

labels = ['Yes', 'No', '?']
sizes = [tak, nie,brak]
plt.title("Republican")
plt.bar(labels, sizes)
plt.show()

W obu przypadkach licza głosów jest dość wyrównana.

# Feature Engineering

In [ ]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn.metrics
import random
from sklearn import manifold
import xgboost as xgb
random.seed(42)
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
from matplotlib import pyplot 

In [ ]:
df=pd.read_csv("congressional_voting_dataset.csv")

### Encoding
W naszych danych kodowanie zmiennych kategorycznych wydaje się nie być dużym wyzwaniem. Głosy na nie oznaczamy jako 0, brak głosu  jako 0.5, a głosy na tak to 1. Podobnie intuicyjnie republikanów oznaczamy jako zera, a demokratów jako jedynki.

In [ ]:
df=df.replace("n", 0)
df=df.replace("y", 1)
df=df.replace("?",  0.5)  
df=df.replace("republican", 0)
df=df.replace("democrat",  1)


### Outliers
Ze względu na kategoryczne wartości w naszych danych, nie widzimy tu outlierów w postaci rzędów, które się szczególnie wyróżniają jedną wartością. Jedyny rząd, który odrzucimy to ten, w którym wartości wszystkich głosowań wynosiły "?" - jest to prawdopodobnie brak danych, bądź dany reprezentant z jakiś osobliwych powodów nie wziął udziału w żadnym głosowaniu. 

In [ ]:
X=df.drop(["political_party"], axis=1)
indexes=[]
colnames=X.columns
for i in range(len(X)):
    for j in range(len(colnames)):
        if X.iloc[i, j]!=0.5:
            break
        if j==len(colnames)-1:
            indexes.append(i)
X=X.drop(248, axis=0)
y=df["political_party"].drop(248, axis=0)
df=df.drop(248, axis=0)
df.to_csv("df_encoded.csv", index=False)

# Feature Selection
Na początku korzystaliśmy z bardziej intycuicyjnych sposobów wyborów cech, a potem zastosowaliśmy metody pokazane na laboratoriach. Spojrzymy najpierw ponownie na macierz korelacji.


In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(df.corr(), annot=True, annot_kws={'size': 8}, fmt='.2f')
plt.show()

Usuniemy dwie zmienne, które w porównaniu z innymi są bardzo mało skorelowane z naszym celem - **water_project_cost_sharing** i **immigration**. Spróbujemy też usunąć zmienną **el_salvador_aid** - mimo, że jest silnie związana z celem, jest także najbardziej skorelowana z innymi zmiennymi objaśniającymi.

In [ ]:
drop=["water_project_cost_sharing", "immigration", "el_salvador_aid"]
X=X.drop(drop, axis=1)

### Model
Naszym modelem bazowym, którego dziś użyjemy, będzie xgboost bez tuningu hiperparametrów.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size = 0.2, random_state = 42)

In [ ]:
xgb_model = xgb.XGBClassifier(objective = "binary:logistic", seed = 42, use_label_encoder=False, verbosity=0)
xgb_model.fit(X_train, y_train)
preds = xgb_model.predict(X_test)
comparison = pd.DataFrame({'actual':y_test, 'predicted':preds})
print("Accuracy: " + str(sum(comparison["actual"] == comparison["predicted"]) / len(comparison) * 100) + "%")

Jak widzimy, nasz model całkiem dobrze sobie radzi z przewidywaniem przynależności do danej partii politycznej - osiąga ponad 96% skuteczności. Na koniec spojrzymy, jak ważne dla niego są poszczególne kolumny - użyjemy do tego wbudowanej funkcji modelu xgb. 

In [ ]:
plt.figure(figsize=(8,8))
pyplot.bar(X.columns, xgb_model.feature_importances_)
plt.xticks(rotation=90)
pyplot.show()

Zgodnie z oczekiwaniami wynikającymi z mapy korelacji, zmienna **physicican_fee_freeze** ma olbrzymi wpływ na predykcje naszego modelu. Na koniec spójrzmy jeszcze, jak wyglądała by skuteczność modelu, gdybyśy wybrali jedynie 5 najważniejszych wg wykresu cech.

In [ ]:
fts=["physician_fee_freeze", "mx_missile", "synfuels_corporation_cutback", "religious_groups_in_schools", "crime"]
X=X[fts]
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size = 0.2, random_state = 42)
xgb_model = xgb.XGBClassifier(objective = "binary:logistic", seed = 42, use_label_encoder=False, verbosity=0)
xgb_model.fit(X_train, y_train)
preds = xgb_model.predict(X_test)
comparison = pd.DataFrame({'actual':y_test, 'predicted':preds})
print("Accuracy: " + str(sum(comparison["actual"] == comparison["predicted"]) / len(comparison) * 100) + "%")


In [ ]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn.metrics
import random
from sklearn import manifold
random.seed(42)


In [ ]:
df=pd.read_csv("congressional_voting_dataset.csv")
df=df.replace("n", 0)
df=df.replace("y", 1)
df=df.replace("?",  0.5)  
df=df.replace("republican", 0)
df=df.replace("democrat",  1)
X=df.drop(["political_party"], axis=1)
indexes=[]
colnames=X.columns
for i in range(len(X)):
    for j in range(len(colnames)):
        if X.iloc[i, j]!=0.5:
            break
        if j==len(colnames)-1:
            indexes.append(i)
X=X.drop(248, axis=0)
y=df["political_party"].drop(248, axis=0)
df.head()

In [ ]:
drop=["water_project_cost_sharing", "immigration", "el_salvador_aid"]
X=X.drop(drop, axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size = 0.2, random_state = 42)
X_best=None

In [ ]:
def selectFeature(selector):
    selector.fit(X_train, y_train)
    X_train_fs = selector.transform(X_train)
    X_test_fs = selector.transform(X_test)
    xgb_model = xgb.XGBClassifier(objective = "binary:logistic", seed = 42, use_label_encoder=False, verbosity=0)
    xgb_model.fit(X_train_fs, y_train)
    yhat = xgb_model.predict(X_test_fs)
    accuracy = accuracy_score(y_test, yhat)
    print('Accuracy: %.2f' % (accuracy*100))


In [ ]:
def selectFeature2(selector):
    clf = Pipeline([
      ('feature_selection', SelectFromModel(selector)),
      ('classification', xgb.XGBClassifier(objective = "binary:logistic", seed = 42, use_label_encoder=False, verbosity=0))
    ])
    clf.fit(X_train, y_train)
    yhat = clf.predict(X_test)
    accuracy = accuracy_score(y_test, yhat)
    print('Accuracy: %.2f' % (accuracy*100))

### CHI2

In [ ]:
for i in range(1,14):
    selectFeature(SelectKBest(chi2, k=i))    


Najwyższe accuracy dla wyboru wszytkich 13 zmiennych

### Recursive Feature Elimination

In [ ]:
from sklearn.feature_selection import RFE
estimator = LogisticRegression()
for i in range(1,14):
    selectFeature(RFE(estimator, n_features_to_select=i, step=1))

 

Najwyższe accuracy przy wyborze min 4 zmiennych

### Mutual info classif

In [ ]:
for i in range(1,14):
    selectFeature(SelectKBest(mutual_info_classif, k=i))

Najwyższe accuracy przy wyborze min 10 zmiennych

### LinearSVC - Linear Support Vector Classification

In [ ]:

# C = 0.005
selectFeature2(LinearSVC(C=0.005, penalty="l1", dual=False))

# C = 0.005
selectFeature2(LinearSVC(C=0.01, penalty="l1", dual=False))

# C = 0.11 
selectFeature2(LinearSVC(C=0.11, penalty="l1", dual=False))

# C = 0.17 
selectFeature2(LinearSVC(C=0.17, penalty="l1", dual=False))

# C = 0.2
selectFeature2(LinearSVC(C=0.2, penalty="l1", dual=False))



In [ ]:
selectFeature2(LinearSVC())#domyślnie C = 1, penalty='l2'

### PolynomialFeatures

In [ ]:
pf = PolynomialFeatures(degree=3)
pf.fit(X_train, y_train)
X_train_fs = pf.transform(X_train)
X_test_fs = pf.transform(X_test)
xgb_model = xgb.XGBClassifier(objective = "binary:logistic", seed = 42, use_label_encoder=False, verbosity=0)
xgb_model.fit(X_train_fs, y_train)
yhat = xgb_model.predict(X_test_fs)
accuracy = accuracy_score(y_test, yhat)
print('Accuracy: %.2f' % (accuracy*100))

In [ ]:
def selectFeaturePF(selector):
    pf = PolynomialFeatures(degree=3)
    pf.fit(X_train, y_train)
    X_train_pf = pf.transform(X_train)
    X_test_pf = pf.transform(X_test)
    selector.fit(X_train_pf, y_train)
    X_train_fs = selector.transform(X_train_pf)
    X_test_fs = selector.transform(X_test_pf)
    xgb_model = xgb.XGBClassifier(objective = "binary:logistic", seed = 42, use_label_encoder=False, verbosity=0)
    xgb_model.fit(X_train_fs, y_train)
    yhat = xgb_model.predict(X_test_fs)
    accuracy = accuracy_score(y_test, yhat)
    print('Accuracy: %.2f' % (accuracy*100))

In [ ]:
for i in range(1,20):
    print(f'i = {i}')
    selectFeaturePF(SelectKBest(chi2, k=i))   
    
print(f'i = {42}')
selectFeaturePF(SelectKBest(chi2, k=42))

In [ ]:
for i in range(1,20):
    print(f'i = {i}')
    selectFeaturePF(SelectKBest(mutual_info_classif, k=i))
    
print(f'i = {37}')
selectFeaturePF(SelectKBest(mutual_info_classif, k=37))

Przy wyborze do k najlepszych zmiennych za pomocą chi2 oraz mutual info classif, accuracy jest większe niż 94.25 dla k min równego 42 i 37.

## Wnioski

Metody osiągają max accuracy 96.55, czyli tyle ile udało nam sie osiągnąć przy samodzielnym wyborze zmiennych. </br>  
Używając niedużej ilości zmiennych udało nam się zatem osiągnąć wynik tylko nieznacznie gorszy od bazowego XGBoosta, który wynosił 97%. Jednak ze względu na małą liczbę rekordów i krótki czas wykonywania algorytmów, nie widzeliśmy sensu w ograniczaniu w tym przypadku liczby kolumn.

In [ ]:
fts=["physician_fee_freeze","synfuels_corporation_cutback", "adoption_of_the_budget_resolution", "education_spending"]
X2=X[fts]
X_train, X_test, y_train, y_test = train_test_split(X2, y, stratify=y, test_size = 0.2, random_state = 42)
xgb_model = xgb.XGBClassifier(objective = "binary:logistic", seed = 42, use_label_encoder=False, verbosity=0)
xgb_model.fit(X_train, y_train)
preds = xgb_model.predict(X_test)
comparison = pd.DataFrame({'actual':y_test, 'predicted':preds})
print("Accuracy: " + str(sum(comparison["actual"] == comparison["predicted"]) / len(comparison) * 100) + "%")

In [ ]:
xgb_tmp=xgb.XGBClassifier(objective = "binary:logistic", seed = 42, use_label_encoder=False, verbosity=0)

In [ ]:
xgb_tmp

In [ ]:
y_train

# Wybór modelu 

In [ ]:
df=pd.read_csv("df_encoded.csv")

Chociaż na we wcześniejszym etapie pracy usunęliśmy z naszych danych zduplikowane rzędy, to podczas pracy z modelami okazuje się, że osiągają one lepsze wyniki kiedy je zostawimy.

In [ ]:
from sklearn.model_selection import train_test_split
X=df.drop(["political_party"], axis=1)
y=df["political_party"]
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,random_state = 42)

Wybraliśmy do przetestowania 3 modele - **SVM**, **Random Forest** i **XGBoost**.

## SVM

In [ ]:
svm_base=SVC(random_state=42)
svm_base.fit(X_train, y_train)
preds=svm_base.predict(X_test)

In [ ]:
svm_base_acc=accuracy_score(preds,y_test)
print("Accuracy SVM z domyślnymi hiperparametrami: " + str(svm_base_acc))

Widzimy, że już domyślny SVM osiąga bardzo dobre accuracy na poziomie ponad 96%. Spróbujemy teraz wykonać tuning hiperparametrów. Ponieważ nasz zbiór danych jest stosunkowo mały, skorzystamy z narzędzia GridSearch.

In [ ]:

svm_tuned=SVC(random_state=42)
c=[]  # wartości parametru C
gamma=[]  #wartości parametru gamma 
for i in range(-4, 5):      # orientacyjne wartości na podstawie informacji znalezionych w internecie
    c.append(10**i)
for i in range(-4, 5):
    gamma.append(10**i)
gamma.append("auto")
gamma.append("scale")
params = [{'C': c,   
           "kernel": ["rbf", "linear", "poly"],
        'gamma': gamma}]
gs_svm=GridSearchCV(svm_tuned, param_grid=params, scoring='accuracy', cv=4, n_jobs=2)
gs_svm.fit(X_train, y_train)
gs_svm.best_params_

In [ ]:
svm_tuned_acc=accuracy_score(gs_svm.predict(X_test),y_test)
print("Accuracy SVM po tuningu hiperparametrów: " + str(svm_tuned_acc))

Jak widzimy, nie udało nam się polepszyć wyniku, a nawet uzyskaliśmy accuracy trochę gorsze. Mimo że w naszej ramce danych znajdują się także domyślne wartości hiperparametrów, to wypadły one gorzej przy kroswalidacji i dlatego algorytm ich nie wybrał. Wydaje mi się, że taka sytuacja zachodzi ze względu na małą liczbę rekordów i duże accuracy naszych modeli.

## XGBoost

In [ ]:
xgb_base = xgb.XGBClassifier(objective = "binary:logistic", seed = 1613, use_label_encoder=False, verbosity=0)
xgb_base.fit(X_train, y_train)
preds=xgb_base.predict(X_test)

In [ ]:
xgb_base_acc=accuracy_score(preds,y_test)
print("Accuracy XGB z domyślnymi hiperparametrami: " + str(xgb_base_acc))

In [ ]:
xgb_tuned=xgb.XGBClassifier(objective = "binary:logistic", seed = 1613, use_label_encoder=False, eval_metric="error")
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

gs_xgb=GridSearchCV(xgb_tuned, param_grid=params, scoring='accuracy', cv=4, n_jobs=2)
gs_xgb.fit(X_train, y_train)
gs_xgb.best_params_

In [ ]:
xgb_tuned_acc=accuracy_score(gs_xgb.predict(X_test), y_test)
print("Accuracy XGB po treningu hiperparametrów: " + str(xgb_tuned_acc))

## Random Forest

In [ ]:
rfc_base = RandomForestClassifier(random_state=16)
rfc_base.fit(X_train, y_train)
preds=rfc_base.predict(X_test)

In [ ]:
rfc_base_acc=accuracy_score(preds,y_test)
print("Accuracy RFC z domyślnymi hiperparametrami: " + str(rfc_base_acc))

In [ ]:
rfc_tuned=RandomForestClassifier(random_state=16)
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 1000, num = 5)] # przykładowe wartości znalezione w internecie 
max_depth = [int(x) for x in np.linspace(5, 55, num = 5)]
max_features= ['auto', 'sqrt', 'log2']
             
params = [{'n_estimators': n_estimators,
        'max_depth': max_depth,
          'max_features': max_features}]
gs_rfc=GridSearchCV(rfc_tuned, param_grid=params, scoring='accuracy', cv=4, n_jobs=2)
gs_rfc.fit(X_train, y_train)
gs_rfc.best_params_

In [ ]:
rfc_tuned_acc=accuracy_score(gs_rfc.predict(X_test), y_test)
print("Accuracy RFC po treningu hiperparametrów: " + str(rfc_tuned_acc))

Jak widać, ani w XGBooście, ani w Random Forest nie udało się uzyskać lepszej niż domyślna accuracy. Co więcej, obydwa bazowe modele osiągają dokładnie ten sam wynik.

# Ocena modeli
## Accuracy score

In [ ]:
scores=[]
labels=[]
scores.append(svm_base_acc)
labels.append("SVM")
scores.append(xgb_base_acc)
labels.append("XGB")
scores.append(rfc_base_acc)
labels.append("RFC")

In [ ]:
pd.DataFrame({"Accuracy Score": scores}, index=labels)

## Confusion matrix
### SVM

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, svm_base.predict(X_test)).ravel()
pd.DataFrame({"Actual positives": [tp, fp], "Actual negatives": [fn, tn]}, index = ["Positive predictions", "Negative predictions"])

### XGB 

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, xgb_base.predict(X_test)).ravel()
pd.DataFrame({"Actual positives": [tp, fp], "Actual negatives": [fn, tn]}, index = ["Positive predictions", "Negative predictions"])

### RFC

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, xgb_base.predict(X_test)).ravel()
pd.DataFrame({"Actual positives": [tp, fp], "Actual negatives": [fn, tn]}, index = ["Positive predictions", "Negative predictions"])

## ROC AUC

In [ ]:
gs_svm
plt.figure(figsize=(12,10))
classifiers = [svm_base, xgb_base, rfc_base]
labels=["SVM", "XGB", "RFC"]
ax = plt.gca()
for i in range(3):
    metrics.plot_roc_curve(classifiers[i], X_test, y_test, ax=ax, name=labels[i])

Na podstawie powyższych wyników zdecydowaliśmy się pozostać przy modelu **XGBoost** bez modyfikacji hiperparametrów czy wyboru zmiennych. W naszym przypadku nawet taki model osiagał bardzo wysokie accuracy, a był przy tym szybki.

# Podsumowanie 
## EDA throwback - które predykcje się udały


In [ ]:
df_tmp=pd.read_csv("congressional_voting_dataset.csv")
df_tmp=df_tmp.replace("n", 0)
df_tmp=df_tmp.replace("y", 1)
df_tmp=df_tmp.replace("?",  0.5)                                                  #żeby uzyskać taki sam wykres jak w eda
df_tmp["political_party"]=df_tmp["political_party"].replace("republican", 0)      #musimy użyć ramki danych z rzędem,
df_tmp["political_party"]=df_tmp["political_party"].replace("democrat", 1)        # który wcześniej wyrzucliśmy

y_tmp=df_tmp["political_party"]
X_tmp=df_tmp.drop(["political_party"], axis=1)


xgb_tmp=xgb.XGBClassifier(objective = "binary:logistic", seed = 1613, use_label_encoder=False, verbosity=0)
df_tmp["predicted_correctly"]=False
for i in range(len(df_tmp)):
    X_train_tmp=X_tmp.drop(i, axis=0)
    y_train_tmp=y_tmp.drop(i, axis=0)
    X_test_tmp=X_tmp.iloc[[i]]
    y_test_tmp=y_tmp[i]
    xgb_tmp.fit(X_train_tmp, y_train_tmp)
    pred_tmp=xgb_tmp.predict(X_test_tmp)[0]
    if y_tmp[i]==pred_tmp:
        df_tmp["predicted_correctly"][i]=True
    

In [ ]:
adist=sklearn.metrics.pairwise_distances(df_tmp.drop(["political_party", "predicted_correctly"], axis=1))
adist

In [ ]:
df_tmp["political_party"]=df_tmp["political_party"].replace(0, "republican")
df_tmp["political_party"]=df_tmp["political_party"].replace(1, "democrat")
adist=np.array(adist)
mds = manifold.MDS(n_components=2, dissimilarity="precomputed", random_state=6)
results = mds.fit(adist)
coords = results.embedding_
fig, ax = plt.subplots(figsize=(12,12))
sns.scatterplot(
    coords[:, 0], coords[:, 1], marker = 'o', hue=df_tmp["political_party"], palette=["red", "blue"], 
    style=df_tmp["predicted_correctly"], markers=["X", "o"], size=df_tmp["predicted_correctly"]
    )
ax.set_title("Voting pattern similarity")

Na koniec wróciliśmy do wykresu z naszego EDA żeby zobaczyć, których z reprezentantów udało przewidzieć się prawidłowo – wykonaliśmy w tym celu predykcje na podstawie reszty ramki danych dla każdego pojedynczego rzędu. Jak widać, błędne predykcje zazwyczaj znajdują się w zróżnicowanym pod względem partii otoczeniu. Jednak duża skuteczność nawet dla nieoczywistych polityków  pokazuje świadczy o jakości XGBoosta, nawet bez specjalnego dopasowywania cech czy tuningu hiperparametrów. 